In [1]:
from Helper.ml_models import *
from Helper.Helper_functions import * 
from torchvision.models.segmentation import *
import gc, sys
import cv2 as cv
from torchvision.utils import draw_segmentation_masks
from tqdm import tqdm


In [2]:
city_label_color_map = np.array([
    [0, 0, 0],       #ID__255, unlabeled
    [128, 64, 128],  #ID__0, road
    [244, 35, 232],  #ID__1, sidewalk
    [70, 70, 70],    #ID__2, building
    [102, 102, 156], #ID__3, wall
    [190, 153, 153], #ID__4, fence
    [153, 153, 153], #ID__5, pole
    [250, 170, 30],  #ID__6, traffic light
    [220, 220, 0],   #ID__7, traffic sign
    [107, 142, 35],  #ID__8, vegetation
    [152, 251, 152], #ID__9, terrain
    [70, 130, 180],  #ID__10, sky
    [220, 20, 60],   #ID__11, person
    [255, 0, 0],     #ID__12, rider
    [0, 0, 142],     #ID__13, car
    [0, 0, 70],      #ID__14, truck
    [0, 60, 100],    #ID__15, bus
    [0, 80, 100],    #ID__16, train
    [0, 0, 230],     #ID__17, motorcycle
    [119, 11, 32],   #ID__18, bicycle
])

In [3]:
duplicates = check_duplicates_in_csv('Daten/CityscapesDaten')
print(f'List of duplicates \n {duplicates} \n')

List of duplicates 
 set() 



In [4]:
k_fold_dataset = K_Fold_Dataset(image_dir='CityscapesDaten/images',
                                            annotation_dir='CityscapesDaten/semantic',
                                            k_fold_csv_dir='Daten/CityscapesDaten',
                                            leave_out_fold=0)
k_fold_dataset.check_for_data_leaks()  

No data leaks found.


In [5]:
image, annotation = k_fold_dataset.train_dataset[0]

print(f'Image shape: {image.shape}, Annotation shape: {annotation.shape}')
print(f'Annotation Value range: {np.unique(annotation)}')

Image shape: torch.Size([3, 520, 520]), Annotation shape: torch.Size([520, 520])
Annotation Value range: [  0   2   5   7   8  10  11  13  18 255]


In [6]:
unique_values = set()
for _, annotation in tqdm(k_fold_dataset.train_dataset, desc='Processing Annotations'):
    unique_values.update(np.unique(annotation))
    if len(unique_values) >= 20:
        break

print(f'Unique values in annotations: {unique_values}')
print(f'Number of unique values in annotations: {len(unique_values)}')


Processing Annotations:   1%|          | 12/2224 [00:00<02:58, 12.36it/s]

Unique values in annotations: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 255}
Number of unique values in annotations: 20


In [ ]:
result_image = draw_segmentation_masks(image,
                                        annotation,
                                        0.5, 
                                        city_label_color_map)